<h1>Superstore time series<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-analysis" data-toc-modified-id="Data-analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data analysis</a></span></li><li><span><a href="#Time-series-data-selection-and-visualization" data-toc-modified-id="Time-series-data-selection-and-visualization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Time series data selection and visualization</a></span></li><li><span><a href="#Model-creation" data-toc-modified-id="Model-creation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model creation</a></span><ul class="toc-item"><li><span><a href="#Simple-dashboard-creation" data-toc-modified-id="Simple-dashboard-creation-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Simple dashboard creation</a></span></li></ul></li></ul></div>

In [56]:
import pandas as pd
pd.set_option('display.max_columns', None)
from skimpy import skim
import plotly.express as px
import plotly.graph_objects as go
import plotly.tools as tls

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss

from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from datetime import timedelta

from sklearn.metrics import mean_squared_error

import dash
import dash_bootstrap_components as dbc
import dash_ag_grid as dag

from dash import html, dash_table
from dash import dcc
from dash.dependencies import Output, Input 
from dash.exceptions import PreventUpdate
from unicodedata import lookup
from dash.dash_table.Format import Format, Group

### Data analysis

In [57]:
df = pd.read_excel('/Users/alexfil/Desktop/git_hub/dash/friday_plotly/Sample - Superstore.xls')

In [58]:
df.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,State/Province,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2021-103800,2021-01-03,2021-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,Texas,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2021-141817,2021-01-05,2021-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,Pennsylvania,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


In [59]:
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 10194  │ │ string      │ 14    │                                                          │
│ │ Number of columns │ 21     │ │ float64     │ 3     │                                                          │
│ └───────────────────┴────────┘ │ int64       │ 2     │                                                          │
│                                │ datetime64  │ 2     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name     ┃ NA   ┃ NA %   ┃ mean   ┃ sd     ┃ p0      ┃ p25    ┃ p50    ┃ p75    ┃ p100    ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩  │
│ │ Row ID          │    0 │      0 │   5100 │   2900 │       1 │   2500 │   5100 │   7600 │   10000 │ ▇▇▇▇▇▇  │  │
│ │ Sales           │    0 │      0 │    230 │    620 │    0.44 │     17 │     54 │    210 │   23000 │    ▇    │  │
│ │ Quantity        │    0 │      0 │    3.8 │    2.2 │       1 │      2 │      3 │      5 │      14 │  ▇▃▂▁   │  │
│ │ Discount        │    0 │      0 │   0.16 │   0.21 │       0 │      0 │    0.2 │    0.2 │     0.8 │ ▇▆   ▁  │  │
│ │ Profit          │    0 │      0 │     29 │    230 │   -6600 │    1.8 │    8.7 │     29 │    8400 │     ▇   │  │
│ └─────────────────┴──────┴────────┴────────┴────────┴─────────┴────────┴────────┴────────┴─────────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name            ┃ NA     ┃ NA %      ┃ first               ┃ last                ┃ frequency        ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩  │
│ │ Order Date             │      0 │         0 │     2021-01-03      │     2024-12-30      │ None             │  │
│ │ Ship Date              │      0 │         0 │     2021-01-07      │     2025-01-05      │ None             │  │
│ └────────────────────────┴────────┴───────────┴─────────────────────┴─────────────────────┴──────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                   ┃ NA     ┃ NA %       ┃ words per row               ┃ total words            ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Order ID                      │      0 │          0 │                           1 │                  10194 │  │
│ │ Ship Mode                     │      0 │          0 │                           2 │                  20388 │  │
│ │ Customer ID                   │      0 │          0 │                           1 │                  10194 │  │
│ │ Customer Name                 │      0 │          0 

### Time series data selection and visualization

In [60]:
ts = df.filter(items=['Order Date', 'Quantity']).set_index('Order Date')

In [61]:
ts.head(5)

,Quantity
Order Date,
2021-01-03,2
2021-01-04,2
2021-01-04,3
2021-01-04,3
2021-01-05,3


In [62]:
ts.sort_index(inplace=True)

In [63]:
ts = ts.resample('M').sum()

In [64]:
ts.head(5)

,Quantity
Order Date,
2021-01-31,306
2021-02-28,159
2021-03-31,597
2021-04-30,536
2021-05-31,504


In [65]:
fig1 = px.line(ts,
              y='Quantity',
#              text='Quantity',
              height=400
              
             )
fig1.update_layout(title_text='Orders qty for total period',
                   margin=dict(l=40,
                               r=40,
                               b=40,
                               t=50
                              )
)
fig1.update_traces(textposition='top right',
                  textfont_size=8)
fig1.show()

In [66]:
def plot_seasonal_decompose(
    x,
    model='additive',
    filt=None,
    period=None,
    two_sided=True,
    extrapolate_trend=0,
    title='Seasonal Decomposition'):
    
    result = seasonal_decompose(x,
                                model=model,
                                filt=filt,
                                period=period,
                                two_sided=two_sided,
                                extrapolate_trend=extrapolate_trend
                               )
    
    fig2 = make_subplots(rows=4,
                        cols=1,
                        subplot_titles=['Observed', 'Trend', 'Seasonal', 'Residuals']
                       )
    fig2.add_trace(
        go.Scatter(x=result.seasonal.index,
                   y=result.observed,
                   mode='lines'),
        row=1,
        col=1
    )

    fig2.add_trace(
            go.Scatter(x=result.trend.index,
                       y=result.trend,
                       mode='lines'),
        row=2,
        col=1
        )

    fig2.add_trace(
            go.Scatter(x=result.seasonal.index,
                       y=result.seasonal,
                       mode='lines'),
        row=3,
        col=1
        )

    fig2.add_trace(
            go.Scatter(x=result.resid.index,
                       y=result.resid,
                       mode='lines'),
        row=4,
        col=1
    )
        
    fig2.update_layout(title='Decomposed time series',
                      showlegend=False,
                      height=400,
                      margin=dict(l=40,
                                  r=40,
                                  b=40,
                                  t=50
                                 )
                      )
    fig2.update_annotations(font_size=10)
    return fig2

In [67]:
fig2 = plot_seasonal_decompose(ts)
fig2.show()

### Model creation

In [68]:
adf_test = adfuller(ts)
 
print('p-value = ' + str(adf_test[1]))

p-value = 0.009353672300957452


In [69]:
train = ts[:'2023-12-31'] 
test = ts['2024-01-31':]

In [70]:
stepwise_fit = auto_arima(ts,
                          start_p=0,
                          d=None,
                          start_q=0,
                          max_p=6,
                          max_d=2,
                          max_q=6,
                          m=12, 
                          start_P=0,
                          start_Q=0,
                          max_P=6,
                          max_D=2,
                          max_Q=6,
                          max_order=25,
                          seasonal=True,
                          D=None,
                          test='adf',
                          seasonal_test='ocsb',
                          trace=True, 
                          error_action='ignore',    
                          suppress_warnings=True,  
                          stepwise=True)           
stepwise_fit.summary() 

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(0,1,0)[12]             : AIC=453.603, Time=0.05 sec
 ARIMA(1,2,0)(1,1,0)[12]             : AIC=441.891, Time=0.37 sec
 ARIMA(0,2,1)(0,1,1)[12]             : AIC=inf, Time=0.15 sec
 ARIMA(1,2,0)(0,1,0)[12]             : AIC=440.072, Time=0.03 sec
 ARIMA(1,2,0)(0,1,1)[12]             : AIC=441.525, Time=0.12 sec
 ARIMA(1,2,0)(1,1,1)[12]             : AIC=inf, Time=0.65 sec
 ARIMA(2,2,0)(0,1,0)[12]             : AIC=425.135, Time=0.16 sec
 ARIMA(2,2,0)(1,1,0)[12]             : AIC=427.103, Time=0.28 sec
 ARIMA(2,2,0)(0,1,1)[12]             : AIC=427.091, Time=0.07 sec
 ARIMA(2,2,0)(1,1,1)[12]             : AIC=inf, Time=0.39 sec
 ARIMA(3,2,0)(0,1,0)[12]             : AIC=425.654, Time=0.12 sec
 ARIMA(2,2,1)(0,1,0)[12]             : AIC=inf, Time=0.15 sec
 ARIMA(1,2,1)(0,1,0)[12]             : AIC=inf, Time=0.08 sec
 ARIMA(3,2,1)(0,1,0)[12]             : AIC=inf, Time=0.13 sec
 ARIMA(2,2,0)(0,1,0)[12] intercept   : AIC=427.133, Time=

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   48
Model:             SARIMAX(2, 2, 0)x(0, 1, 0, 12)   Log Likelihood                -209.568
Date:                            Sun, 14 Jul 2024   AIC                            425.135
Time:                                    11:29:21   BIC                            429.714
Sample:                                01-31-2021   HQIC                           426.697
                                     - 12-31-2024                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.9946      0.131     -7.603      0.000      -1.251      -0.738
ar.L2         -0.6278      0.136     -4.615      0.000      -0.894      -0.361
sigma2      1.287e+04   3754.304      3.427      0.001    5508.801    2.02e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.56   Jarque-Bera (JB):                 0.72
Prob(Q):                              0.46   Prob(JB):                         0.70
Heteroskedasticity (H):               2.85   Skew:                            -0.20
Prob(H) (two-sided):                  0.10   Kurtosis:                         2.42
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [71]:
start = len(train)
end = len(train) + len(test)-1

In [72]:
model_tuned = SARIMAX(ts,
                      order = (2, 2, 0),  
                      seasonal_order =(0, 1, 0, 12)
                     ) 
result = model_tuned.fit() 
result.summary() 

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.36674D+00    |proj g|=  4.38402D-02

At iterate    5    f=  4.36601D+00    |proj g|=  9.74700D-05

At iterate   10    f=  4.36601D+00    |proj g|=  6.35649D-04

At iterate   15    f=  4.36600D+00    |proj g|=  2.18989D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     19     22      1     0     0   1.887D-06   4.366D+00
  F =   4.3659929407306297     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                           Quantity   No. Observations:                   48
Model:             SARIMAX(2, 2, 0)x(0, 1, 0, 12)   Log Likelihood                -209.568
Date:                            Sun, 14 Jul 2024   AIC                            425.135
Time:                                    11:29:22   BIC                            429.714
Sample:                                01-31-2021   HQIC                           426.697
                                     - 12-31-2024                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.9946      0.131     -7.603      0.000      -1.251      -0.738
ar.L2         -0.6278      0.136     -4.615      0.000      -0.894      -0.361
sigma2      1.287e+04   3754.304      3.427      0.001    5508.801    2.02e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.56   Jarque-Bera (JB):                 0.72
Prob(Q):                              0.46   Prob(JB):                         0.70
Heteroskedasticity (H):               2.85   Skew:                            -0.20
Prob(H) (two-sided):                  0.10   Kurtosis:                         2.42
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [73]:
predictions_tuned = result.predict(start, end)

In [74]:
predictions_tuned.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 12 entries, 2024-01-31 to 2024-12-31
Freq: M
Series name: predicted_mean
Non-Null Count  Dtype  
--------------  -----  
12 non-null     float64
dtypes: float64(1)
memory usage: 192.0 bytes


In [75]:
print('MSE: ', mean_squared_error(test.dropna(), predictions_tuned))
print('RMSE: ', np.sqrt(mean_squared_error(test.dropna(), predictions_tuned)))

MSE:  16455.76960490624
RMSE:  128.2800436736215


In [76]:
predictions_tuned = result.predict(start, end+12)

In [77]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=train.index, y=train['Quantity'],
                    mode='lines',
                    name='train'))
fig3.add_trace(go.Scatter(x=test.index, y=test['Quantity'],
                    mode='lines',
                    name='test'))
fig3.add_trace(go.Scatter(x=predictions_tuned.index, y=predictions_tuned,
                    mode='lines',
                    name='prediction'))
fig3.update_layout(
    title='Prediction for time series',
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1),
    margin=dict(l=40,
                r=40,
                b=40,
                t=50
                )
)


fig3.show()

#### Simple dashboard creation

In [78]:
app = dash.Dash(__name__,
                external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME, dbc.icons.BOOTSTRAP])

In [79]:
app.layout = html.Div([
    html.H2('Time series visualization for superstore dataset'),
    
    dbc.Row([
       dbc.Col([
            dcc.Graph(figure=fig1)
        ], lg=6),
       
       dbc.Col([
            dcc.Graph(figure=fig2)
        ], lg=6),
       
   ]),
    html.H2('Orders quantity predictions with ARIMA model'),
    
    dbc.Row([
        dcc.Graph(id='graph-with-slider')
    ]
    ),
    html.H2('Select of forecast period:'),
    dbc.Row([
        dcc.Slider(0, 12,
        step=1,
        value=0,
        marks={0: '0M', 1: '1M', 2: '2M', 3: '3M', 4: '4M', 5: '5M', 6: '6M',
               7: '7M', 8: '8M', 9: '9M', 10: '10M', 11: '11M', 12: '12M'},
        id='year-slider'
    )
    ], className='slider-track')
   
   ], className='body_div')      

In [80]:
@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'))

def update_figure(selected_period):
    predictions_tuned = result.predict(start, end+selected_period)

    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=train.index, y=train['Quantity'],
                    mode='lines',
                    name='train data'))
    fig3.add_trace(go.Scatter(x=test.index, y=test['Quantity'],
                    mode='lines',
                    name='test data'))
    fig3.add_trace(go.Scatter(x=predictions_tuned.index, y=predictions_tuned,
                    mode='lines',
                    name='prediction data'))
    fig3.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
       ),
        height=400,
        margin=dict(l=40,
                    r=40,
                    b=40,
                    t=50
                   )
    )

    return fig3

In [81]:
if __name__ == '__main__':
    app.run_server(debug=False, port=8015, host='127.0.0.1', use_reloader=False)

Dash is running on http://127.0.0.1:8015/

Dash is running on http://127.0.0.1:8015/

Dash is running on http://127.0.0.1:8015/

Dash is running on http://127.0.0.1:8015/

Dash is running on http://127.0.0.1:8015/

Dash is running on http://127.0.0.1:8015/

Dash is running on http://127.0.0.1:8015/

Dash is running on http://127.0.0.1:8015/

 * Serving Flask app '__main__'
 * Debug mode: off
